###  Preparación de los datos

El objetivo de este script es prepara la información total en los formatos requeridos para la ejecución del modelo.

Considerando que existen variables numericas, categoricas ordinales y nominales.:

* Categoricas ordinales:  La variables categoricas tienen un orden específico a través del tipo "Category", si este orden no cumple con el entrenamiento no se puede aprovechar el modelo. Adicionalmente, el dataset puede no tener todas las categorias utilizadas en el entrenamiento lo que genera problemas al procesar la base y resultados inesperados

* Nominales: Se convierten en variables dummies, a través del proceso one hot encoder. Para garantizar el orden y cantidad de columnas generadas se carga el objeto creado en la fase de entrenamiento del modelo y se aplica sobre la base general a procesar.

* Se transforma y ejecuta la nueva base por tandas, se captura el valor pronosticado y real
* Se evaluan métricas de evaluación sobre la base general


* No olvidar procesar el estrato

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import random

pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from mlxtend.evaluate import lift_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import shap
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score,roc_auc_score, f1_score, plot_roc_curve, plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime

from joblib import Parallel, delayed, dump, load
import gc
import pickle

### Carga elementos generadores

In [3]:
path_data = 'modeloEntrenado_new/'
file= f'GtableCategorical.csv'

In [4]:
GtableCategorical = pd.read_csv(f'{path_data}{file}', sep=',', decimal='.').set_index('Unnamed: 0')

In [5]:
#path_data = ''
file= f'GtableNominales.csv'

In [6]:
GtableNominales = pd.read_csv(f'{path_data}{file}', sep=',', decimal='.').set_index('Unnamed: 0')

In [7]:
GtableNominales

,DEPARTAMENTO,COD_PLAN_ACTUAL,MARKETING_NAME,MANUFACTURER
Unnamed: 0,,,,
0,BOGOTA D C,25017-4103,Galaxy A21s,Samsung
1,ANTIOQUIA,25867-4103,Galaxy A30s,HUAWEI
2,CUNDINAMARCA,25522-6734,HUAWEI Y9 Prime 2019,Motorola
3,VALLE DEL CAUCA,24231-4103,Redmi Note 8,Redmi
4,BOYACA,23863-4103,Redmi Note 9,Apple


In [8]:
ruta_objeto = f"{path_data}class_ohe.pkl"
#class_ohe = load(ruta_objeto)
class_ohe = load(ruta_objeto)

In [9]:
colCatNom=['DEPARTAMENTO','TIPO_MUNICIPIO','MARKETING_NAME','MANUFACTURER','COD_PLAN_ACTUAL',
           'OPERADOR_4GBEST_DW','OPERADOR_4GBEST_UP','OPERADOR_4GBEST_LA',
           'OPERADOR_4GBEST_DB','OPERADOR_4GCLARO','OPERADOR_3GBEST_DW', 'OPERADOR_3GBEST_UP', 'OPERADOR_3GBEST_LA',
           'OPERADOR_3GBEST_DB', 'OPERADOR_3GCLARO', 'OPERADOR_CLARO']
colBinning=['DEPARTAMENTO','COD_PLAN_ACTUAL','MARKETING_NAME','MANUFACTURER']
colCatOrd=['COMPORTAMIENTO_PAGO','MIN_CALIFICACION_CREDITO','MAX_CALIFICACION_CREDITO',
           'TECH_EQUIPO','TECNOLOGIA_EQUIPO',
           'TENDENCIA_CHURN_GSM','TENDENCIA_CHURN_LTE','TENDENCIA_CHURN_UMTS',
           'TENDENCIA_DOWN_4G_CLARO','TENDENCIA_DOWN_4G_BEST',
           'TENDENCIA_LLAMADAS_GSM', 'TENDENCIA_LLAMADAS_UTMS','TENDENCIA_LLAMADAS_LTE',
           'CL_TASA_CHURN','CL_CHURN_LATENCIA_4G']

In [10]:
### cargar modelo
ruta_modelo = f"{path_data}modelLGBM2.pkl"
modelLGBM2 = load(ruta_modelo)

In [11]:
## variables para guardar resultados
predicciones={}
indexes={}
reales={}

### Tipos de Datos categoricos ordinales

In [12]:
for col in colCatOrd:
    GtableCategorical[col] = GtableCategorical[col].astype('category')

In [13]:
GtableCategorical

,COMPORTAMIENTO_PAGO,MIN_CALIFICACION_CREDITO,MAX_CALIFICACION_CREDITO,TECH_EQUIPO,TECNOLOGIA_EQUIPO,TENDENCIA_CHURN_GSM,TENDENCIA_CHURN_LTE,TENDENCIA_CHURN_UMTS,TENDENCIA_DOWN_4G_CLARO,TENDENCIA_DOWN_4G_BEST,TENDENCIA_LLAMADAS_GSM,TENDENCIA_LLAMADAS_UTMS,TENDENCIA_LLAMADAS_LTE,CL_TASA_CHURN,CL_CHURN_LATENCIA_4G
Unnamed: 0,,,,,,,,,,,,,,,
0,0,SIN_ESTUDIO,SIN_ESTUDIO,0,0,0,0,0,0,0,0,0,0,0,0
1,ALTO RIESGO,P-PREPAGO,P-PREPAGO,2G,2G,decreasing,decreasing,decreasing,decreasing,decreasing,decreasing,decreasing,no trend,B-B,B-B
2,REGULAR,7-POSPAGO,7-POSPAGO,3G,3G,no trend,no trend,no trend,no trend,no trend,no trend,no trend,0,B-A,B-A
3,BUENO,5-POSPAGO,5-POSPAGO,4G,LTE,increasing,increasing,increasing,increasing,increasing,increasing,increasing,0,A-B,0
4,MUY BUENO,E-POSPAGO,E-POSPAGO,0,4G,0,0,0,0,0,0,0,0,A-A,0
5,NUEVO,D-POSPAGO,D-POSPAGO,0,0,0,0,0,0,0,0,0,0,0,0
6,0,C-POSPAGO,C-POSPAGO,0,0,0,0,0,0,0,0,0,0,0,0
7,0,B-POSPAGO,B-POSPAGO,0,0,0,0,0,0,0,0,0,0,0,0
8,0,A-POSPAGO,A-POSPAGO,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
GtableCategorical['COMPORTAMIENTO_PAGO'].cat.reorder_categories(['0','ALTO RIESGO','REGULAR','BUENO','MUY BUENO','NUEVO'], inplace=True)
GtableCategorical['TENDENCIA_CHURN_GSM'].cat.reorder_categories(['0','decreasing','no trend','increasing'], inplace=True)
GtableCategorical['TENDENCIA_CHURN_LTE'].cat.reorder_categories(['0','decreasing','no trend','increasing'], inplace=True)
GtableCategorical['TENDENCIA_CHURN_UMTS'].cat.reorder_categories(['0','decreasing','no trend','increasing'], inplace=True)
GtableCategorical['TENDENCIA_DOWN_4G_CLARO'].cat.reorder_categories(['0','decreasing','no trend','increasing'], inplace=True)
GtableCategorical['TENDENCIA_DOWN_4G_BEST'].cat.reorder_categories(['0','decreasing','no trend','increasing'], inplace=True)
GtableCategorical['TENDENCIA_LLAMADAS_GSM'].cat.reorder_categories(['0','decreasing','no trend','increasing'], inplace=True)
GtableCategorical['TENDENCIA_LLAMADAS_UTMS'].cat.reorder_categories(['0','decreasing','no trend','increasing'], inplace=True)
GtableCategorical['TENDENCIA_LLAMADAS_LTE'].cat.reorder_categories(['0','no trend'], inplace=True)
GtableCategorical['MIN_CALIFICACION_CREDITO'].cat.reorder_categories(['SIN_ESTUDIO','P-PREPAGO','7-POSPAGO','5-POSPAGO','E-POSPAGO','D-POSPAGO','C-POSPAGO','B-POSPAGO','A-POSPAGO'], inplace=True)
GtableCategorical['MAX_CALIFICACION_CREDITO'].cat.reorder_categories(['SIN_ESTUDIO','P-PREPAGO','7-POSPAGO','5-POSPAGO','E-POSPAGO','D-POSPAGO','C-POSPAGO','B-POSPAGO','A-POSPAGO'], inplace=True)
GtableCategorical['TECH_EQUIPO'].cat.reorder_categories(['0','2G','3G','4G'], inplace=True)
GtableCategorical['TECNOLOGIA_EQUIPO'].cat.reorder_categories(['0','2G','3G','LTE','4G'], inplace=True)
GtableCategorical['CL_TASA_CHURN'].cat.reorder_categories(['0','B-B','B-A','A-B','A-A'], inplace=True)
GtableCategorical['CL_CHURN_LATENCIA_4G'].cat.reorder_categories(['0','B-B','B-A'], inplace=True)

In [15]:
GtableNominales

,DEPARTAMENTO,COD_PLAN_ACTUAL,MARKETING_NAME,MANUFACTURER
Unnamed: 0,,,,
0,BOGOTA D C,25017-4103,Galaxy A21s,Samsung
1,ANTIOQUIA,25867-4103,Galaxy A30s,HUAWEI
2,CUNDINAMARCA,25522-6734,HUAWEI Y9 Prime 2019,Motorola
3,VALLE DEL CAUCA,24231-4103,Redmi Note 8,Redmi
4,BOYACA,23863-4103,Redmi Note 9,Apple


### Cargar datos

In [20]:
start = datetime.now()
path_data = ''
file= f'GAAVANZADA.TTA_ANALITICA_CAMBIO_EQUI_COMPLETA_FINANCIA_202111.csv'
df = pd.read_csv(f'{path_data}{file}', sep='|', decimal='.',error_bad_lines=False)

Columns (0,48,60,61,64,65,66,68,69,70,71,72,73,75,77,79,81,90,92,94,96,98,107,108,109,110,126,127) have mixed types.Specify dtype option on import or set low_memory=False.


In [21]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
EDAD_MESES,6269831.00,556.13,989.02,-95725.94,366.35,482.65,633.84,24250.94
CANTIDAD_LINEAS,6313372.00,1.10,0.41,1.00,1.00,1.00,1.00,234.00
TIPO_IDENTIFICACION,6313372.00,1.01,0.16,1.00,1.00,1.00,1.00,4.00
ANTIGUEDAD_MESES,6313059.00,45.71,49.67,0.13,8.47,26.46,67.07,221.09
VLR_CFM,6313372.00,45209.70,22746.24,0.00,31631.24,38301.04,51661.57,3026662.20
CANT_MB_INC_PLAN,6313372.00,2576268.92,16710730.01,0.00,15360.00,20480.00,30720.00,3499999965.00
NUM_UPGRADES,6313372.00,0.02,0.14,0.00,0.00,0.00,0.00,5.00
NUM_DOWNGRADES,6313372.00,0.07,0.27,0.00,0.00,0.00,0.00,12.00
NUM_PARALELO,6313372.00,0.00,0.02,0.00,0.00,0.00,0.00,2.00
FACTURADO_CFM_MM1,6313372.00,40641.62,33261.56,-490477.46,29900.00,39900.00,55900.01,3636500.00


In [22]:
df.isnull().sum()

IDENTIFICACION                         0
EDAD_MESES                         43541
CANTIDAD_LINEAS                        0
TIPO_IDENTIFICACION                    0
ANTIGUEDAD_MESES                     313
VLR_CFM                                0
CANT_MB_INC_PLAN                       0
NUM_UPGRADES                           0
NUM_DOWNGRADES                         0
NUM_PARALELO                           0
FACTURADO_CFM_MM1                      0
FACTURADO_CFM_MM2                      0
FACTURADO_CFM_MM3                      0
FACTURADO_CFM_MM4                      0
FACTURADO_CFM_MM5                      0
FACTURADO_CFM_MM6                      0
FACTURADO_ADICIONALES_MM1              0
FACTURADO_ADICIONALES_MM2              0
FACTURADO_ADICIONALES_MM3              0
FACTURADO_ADICIONALES_MM4              0
FACTURADO_ADICIONALES_MM5              0
FACTURADO_ADICIONALES_MM6              0
NUM_SUSPENSIONES                       0
MAX_EDAD_MORA_MM1                      0
MAX_EDAD_MORA_MM

In [23]:
df['MANUFACTURER_OR_APPLICANT_MM2'].value_counts()

Series([], Name: MANUFACTURER_OR_APPLICANT_MM2, dtype: int64)

In [24]:
#desc=df.describe()
dup=df[["IDENTIFICACION","MANUFACTURER_OR_APPLICANT_MM1"]].groupby(by=["IDENTIFICACION"], dropna=False).count()
dup.sort_values('MANUFACTURER_OR_APPLICANT_MM1', ascending=False).head(10)
dup.reset_index().groupby(by=["MANUFACTURER_OR_APPLICANT_MM1"]).count()
#  Numero Identificaciones
dup.reset_index().groupby(by=["MANUFACTURER_OR_APPLICANT_MM1"]).count().sum()


IDENTIFICACION    6313371
dtype: int64

In [25]:
len(df)

6313372

### Etiquetas datos

In [26]:
# llenar nan
df['MANUFACTURER_OR_APPLICANT_MM1'].fillna("SinEquipo", inplace=True)
df['MANUFACTURER_OR_APPLICANT_MM2'].fillna("SinEquipo", inplace=True)
df[['MANUFACTURER_OR_APPLICANT_MM1']].value_counts()
df[['MANUFACTURER_OR_APPLICANT_MM1']].isna().sum()
df[['MANUFACTURER_OR_APPLICANT_MM2']].value_counts()
df[['MANUFACTURER_OR_APPLICANT_MM2']].isna().sum()
E_MF_MM1=df[['IDENTIFICACION','MANUFACTURER_OR_APPLICANT_MM1']]
E_MF_MM2=df[['IDENTIFICACION','MANUFACTURER_OR_APPLICANT_MM2']]

In [27]:
E_MF_MM1_T=pd.pivot_table(E_MF_MM1, index='IDENTIFICACION', columns='MANUFACTURER_OR_APPLICANT_MM1', aggfunc=len, fill_value=0)
E_MF_MM2_T=pd.pivot_table(E_MF_MM2, index='IDENTIFICACION', columns='MANUFACTURER_OR_APPLICANT_MM2', aggfunc=len, fill_value=0)
cols=list(set(np.concatenate((E_MF_MM1_T.columns.values,E_MF_MM2_T.columns.values))))
#E_MF_MM1_T['HMD Global Oy'].head(2)
#(E_MF_MM1_T['HMD Global Oy']+E_MF_MM2_T['HMD Global Oy']).head(2)
E_MF=(E_MF_MM1_T+E_MF_MM2_T)
E_MF["Total"]=E_MF.drop('SinEquipo',1).sum(axis=1)
for col in cols+['Total'] :
    E_MF[col]=np.where(E_MF[col]>0,1,0)

### Reducción por IDENTIFICICACIÓN

In [28]:
# ### Reducción por IDENTIFICICACIÓN
variableRechazadas=['MSISDN','TAC','CO_ID','TELE_NUMB','MANUFACTURER_OR_APPLICANT_MM1','MANUFACTURER_OR_APPLICANT_MM2','SITIO']
tablaAnalisis=df.copy()
tablaAnalisis['ESTRATO'].value_counts()
tablaAnalisis['ESTRATO'].isna().sum()
tablaAnalisis['ESTRATO']=tablaAnalisis['ESTRATO'].fillna(0)
tablaAnalisis['ESTRATO']=np.where(tablaAnalisis['ESTRATO']=='nulo',0,tablaAnalisis['ESTRATO'])
tablaAnalisis['ESTRATO']=tablaAnalisis['ESTRATO'].astype(int)
tablaAnalisis=tablaAnalisis.drop(columns=variableRechazadas).drop_duplicates()

In [29]:
tablaAnalisis.count()
tablaAnalisis.set_index('IDENTIFICACION', inplace=True)

In [30]:
tablaAnalisis=tablaAnalisis.fillna(0)

In [31]:
colCategoricas=tablaAnalisis.select_dtypes('object').columns
colNumericas=tablaAnalisis.select_dtypes(exclude='object').columns

In [32]:
for col in colCategoricas:
        #dm_traindf[dm_class_input].applymap(str)
        tablaAnalisis[col]=tablaAnalisis[[col]].applymap(str) 

## Nominales 
se puede hacer una vez al inicio

In [33]:
## hacer binning
for col in colBinning:    
    auxBin=GtableNominales[col].unique()    
    tablaAnalisis[col]=np.where(tablaAnalisis[col].isin(auxBin), tablaAnalisis[col] , 'OTRO')

In [34]:
colsNotNoms=tablaAnalisis.drop(columns=colCatNom).columns.values.tolist()

In [35]:
colDummies=class_ohe.get_feature_names(colCatNom)

In [36]:
colsFinal=colsNotNoms
colsFinal.extend(colDummies)

#### Predicciones

In [37]:
#num_registros=len(tablaAnalisis_base)
num_registros=len(tablaAnalisis)

In [38]:
tablaAnalisis_base=tablaAnalisis

In [39]:
predicciones={}
indexes={}
reales={}

In [40]:
for i in range(0,6):
    print(i)

0
1
2
3
4
5


In [41]:
len(tablaAnalisis_base)

6313372

In [42]:
imax=1+(len(tablaAnalisis_base)//(10**6))
imax

7

In [43]:
for i in range(0,imax):
    print(i)
    i_inicial=i*1000000
    i_final=min((i+1)*1000000,num_registros)
    tablaAnalisis=tablaAnalisis_base.iloc[i_inicial:i_final]
    tablaAnalisis_class_ohe = class_ohe.transform(tablaAnalisis[colCatNom]).toarray()
    tablaAnalisisFinal=pd.DataFrame(np.concatenate((tablaAnalisis.drop(columns=colCatNom), tablaAnalisis_class_ohe), axis=1),index=tablaAnalisis.index, columns=colsFinal)
    tablaAnalisisComp=tablaAnalisisFinal.join(E_MF)
    #tipos de dato
    for col in colCatOrd:
        tablaAnalisisComp[col]=tablaAnalisisComp[col].astype(GtableCategorical.dtypes[col])
    for col in colCatOrd:
        tablaAnalisisComp[col]=tablaAnalisisComp[col].cat.codes   
    for col in colDummies:
        tablaAnalisisComp[col]=tablaAnalisisComp[col].astype('int64')
    for col in colNumericas:
        tablaAnalisisComp[col]=tablaAnalisisComp[col].astype(tablaAnalisis.dtypes[col])
    # Prediccion final
    pred_final=modelLGBM2.predict(tablaAnalisisComp[colsFinal])
    predicciones[i]=pred_final
    indexes[i]=tablaAnalisisComp.index.values
    reales[i]= tablaAnalisisComp[['Total']]
    
    del tablaAnalisisFinal
    del tablaAnalisisComp        
    gc.collect()
        

0
1
2
3
4
5
6


In [44]:
reales[0].head(2)

,Total
IDENTIFICACION,
1063152364,0
1030531294,0


In [45]:
type(predicciones[0])

numpy.ndarray

In [46]:
## Unir respuesta
reales_total=reales[0]
predicciones_total=predicciones[0]

In [47]:
for i in range(1,imax):
    reales_total=reales_total.append(reales[i])
    predicciones_total=np.append(predicciones_total,predicciones[i])    

In [48]:
## Aqui vamos

In [49]:
len(reales_total)

6313372

In [50]:
len(predicciones_total)

6313372

In [51]:
predicciones_total

array([0.07641324, 0.35378912, 0.54200555, ..., 0.41975336, 0.5136366 ,
       0.0202286 ])

In [52]:
tablaAnalisis_base.head(2)

,EDAD_MESES,CANTIDAD_LINEAS,TIPO_IDENTIFICACION,ANTIGUEDAD_MESES,VLR_CFM,CANT_MB_INC_PLAN,NUM_UPGRADES,NUM_DOWNGRADES,NUM_PARALELO,FACTURADO_CFM_MM1,FACTURADO_CFM_MM2,FACTURADO_CFM_MM3,FACTURADO_CFM_MM4,FACTURADO_CFM_MM5,FACTURADO_CFM_MM6,FACTURADO_ADICIONALES_MM1,FACTURADO_ADICIONALES_MM2,FACTURADO_ADICIONALES_MM3,FACTURADO_ADICIONALES_MM4,FACTURADO_ADICIONALES_MM5,FACTURADO_ADICIONALES_MM6,NUM_SUSPENSIONES,MAX_EDAD_MORA_MM1,MAX_EDAD_MORA_MM2,MAX_EDAD_MORA_MM3,MAX_EDAD_MORA_MM4,CONSUMO_TOTAL_MB_MM1,CONSUMO_TOTAL_MB_MM2,CONSUMO_TOTAL_MB_MM3,CONSUMO_TOTAL_MB_MM4,MIN_ENTRANTES_MM1,MIN_SALIENTES_MM1,MIN_ENTRANTES_MM2,MIN_SALIENTES_MM2,MIN_ENTRANTES_MM3,MIN_SALIENTES_MM3,MIN_ENTRANTES_MM4,MIN_SALIENTES_MM4,NUM_RETENCION,DIAS_IMEI_ACTUAL,DIAS_ANTIGUEDAD_IMEI,CANTIDAD_CAMBIOS_EQUIPO,COD_MIN_CALIFICACION_CREDITO,COD_MAX_CALIFICACION_CREDITO,DEPARTAMENTO,TIPO_MUNICIPIO,INICIO_CICLO,ESTRATO,SCORE_VOLUNTARIO,SCORE_INVOLUNTARIO,SCORE_PORTABILIDAD,COMPORTAMIENTO_PAGO,MIN_CALIFICACION_CREDITO,MAX_CALIFICACION_CREDITO,TECH_EQUIPO,TECNOLOGIA_EQUIPO,MARKETING_NAME,MANUFACTURER,COD_PLAN_ACTUAL,ANTIUEDAD_PLAN_ACTUAL_MESES,TENDENCIA_CHURN_GSM,TENDENCIA_CHURN_LTE,TENDENCIA_CHURN_UMTS,TENDENCIA_DOWN_4G_CLARO,TENDENCIA_DOWN_4G_BEST,OPERADOR_4GBEST_DW,DOWNLOAD_KBPS_4GBEST_DW,OPERADOR_4GBEST_UP,UPLOAD_KBPS_4GBEST_UP,OPERADOR_4GBEST_LA,LATENCY_4GBEST_LA,OPERADOR_4GBEST_DB,DBM_4GBEST_DB,OPERADOR_4GCLARO,DOWNLOAD_KBPS_4GCLARO,UPLOAD_KBPS_4GCLARO,LATENCY_4GCLARO,DBM_4GCLARO,DIFERENCIA_MEJOR_4G_DOWN,DIFERENCIA_MEJOR_4G_UP,DIFERENCIA_MEJOR_4G_LAT,DIFERENCIA_MEJOR_4G_DBM,OPERADOR_3GBEST_DW,DOWNLOAD_KBPS_3GBEST_DW,OPERADOR_3GBEST_UP,UPLOAD_KBPS_3GBEST_UP,OPERADOR_3GBEST_LA,LATENCY_3GBEST_LA,OPERADOR_3GBEST_DB,DBM_3GBEST_DB,OPERADOR_3GCLARO,DOWNLOAD_KBPS_3GCLARO,UPLOAD_KBPS_3GCLARO,LATENCY_3GCLARO,DBM_3GCLARO,DIFERENCIA_MEJOR_3G_DOWN,DIFERENCIA_MEJOR_3G_UP,DIFERENCIA_MEJOR_3G_LAT,DIFERENCIA_MEJOR_3G_DBM,OPERADOR_CLARO,TENDENCIA_LLAMADAS_GSM,TENDENCIA_LLAMADAS_UTMS,TENDENCIA_LLAMADAS_LTE,GSM,LTE,UMTS,SOLO_DATOS,SOLO_VOZ,VOZ_DATOS,LINEAS_TOTALES,CANTIDAD_USER,CANT_POST,CANT_PREP,TASA_CHURN,CONTEO_RECLAMACIONES,TASA_SARA,TASA_RECLAMA,W_TASA_CHURN,CL_TASA_CHURN,CL_CHURN_LATENCIA_4G,COMPRAS_TERMINALES_MM1,COMPRAS_TERMINALES_MM2,COMPRAS_TERMINALES_MM3,COMPRAS_TERMINALES_MM4,COMPRAS_TECNOLOGIA_MM1,COMPRAS_TECNOLOGIA_MM2,COMPRAS_TECNOLOGIA_MM3,COMPRAS_TECNOLOGIA_MM4
IDENTIFICACION,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1063152364,0.00,1,1,2.40,46653.99,30720,0,0,0,55900.01,55900.01,0.00,0.00,0.00,0.00,7626.58,1399.62,0.00,0.00,0.00,0.00,1,30,30.00,0.00,0.00,8556.00,19645.00,0.00,0.00,172.00,302.00,120.00,250.00,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,9,9,OTRO,CABECERA,20.00,1,0.07,0.05,0.04,NUEVO,SIN_ESTUDIO,SIN_ESTUDIO,3G,3G,OTRO,OTRO,25522-6734,2.42,no trend,no trend,no trend,no trend,decreasing,MOVISTAR,29797.01,MOVISTAR,21479.76,MOVISTAR,20.97,CLARO,-85.23,CLARO,21719.66,19140.75,38.21,-85.23,-0.27,-0.11,0.82,-0.00,CLARO,7633.25,MOVISTAR,9975.00,MOVISTAR,23.00,WOM,-81.00,CLARO,7633.25,8343.62,58.62,-82.50,0.00,-0.16,1.55,0.02,CLARO,no trend,no trend,no trend,191.00,7.00,42.00,23.00,7.00,210.00,240.00,7581.22,1783.56,5797.67,0.03,28.00,0.00,0.00,0.02,A-A,B-B,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1030531294,420.39,1,1,17.80,33300.43,15360,0,0,0,39900.00,39900.00,39900.00,39900.00,39900.00,39900.00,0.00,0.00,0.00,261.71,0.00,162.08,0,0,0.00,0.00,0.00,7874.00,5752.00,4443.00,6390.00,131.00,95.00,216.00,261.00,169.00,214.00,132.00,281.00,0,712.00,712.00,1.00,3,3,BOGOTA D C,CABECERA,7.00,5,0.03,0.01,0.03,REGULAR,B-POSPAGO,B-POSPAGO,4G,4G,OTRO,Samsung,24231-4103,1.10,no trend,no trend,no trend,no trend,no trend,TIGO,59657.90,TIGO,28119.80,TIGO,18.90,CLARO,-76.10,CLARO,35414.36,19464.22,34.82,-76.10,-0.41,-0.31,0.84,-0.00,TIGO,23814.00,TIGO,33637.00,TIGO,32.00,AVANTEL,-5.00,CLARO,18300.42,5588.42,46.25,-70.25,-0.23,-0.83,0.45,13.05,CLARO,no trend,no trend,no trend,2.00,0.00,48.00,3.00

In [53]:
reales_total.head(2)

,Total
IDENTIFICACION,
1063152364,0
1030531294,0


In [54]:
#unir respuestas
reales_total['predicciones']=predicciones_total

In [55]:
reales_total['Clase']=np.where(reales_total['predicciones']>=0.5,1,0)

In [56]:
reales_total.drop(columns=['Total'],inplace=True)

In [57]:
archivoFinal=reales_total.merge(tablaAnalisis_base[['TIPO_IDENTIFICACION']],left_index=True,right_index=True, how='left')

In [58]:
archivoFinal=archivoFinal[['TIPO_IDENTIFICACION','predicciones','Clase']]

In [59]:
archivoFinal.head(2)

,TIPO_IDENTIFICACION,predicciones,Clase
IDENTIFICACION,,,
0,1,0.30,0
0,1,0.30,0


### GenerarArchivos Entrega

In [60]:
archivoFinal.reset_index(inplace=True)

In [61]:
archivoFinal.columns=['IDENTIFICACION', 'TIPO IDENTIFICACION', 'PROBABILIDAD DE CAMBIO', 'CLASE']

In [62]:
archivoFinal.to_csv('BASE_PREDICCION.csv',sep='|', index=False)

### Percentiles

In [63]:
decil=archivoFinal['PROBABILIDAD DE CAMBIO'].quantile(0.9)

In [64]:
BASE_CONTROL=archivoFinal[archivoFinal['PROBABILIDAD DE CAMBIO']>decil]

In [65]:
df_permutated = BASE_CONTROL.sample(frac=1)

In [66]:
len(df_permutated)

631338

In [67]:
baseControSize = int(len(df_permutated)*0.1)
#train_end = int(len(df_permutated)*train_size)
baseControSize

63133

In [68]:
BASE_CONTROL = df_permutated[:baseControSize]
BASE_GESTION = df_permutated[baseControSize:]

In [69]:
len(BASE_CONTROL)

63133

In [70]:
len(BASE_GESTION)

568205

In [71]:
BASE_CONTROL.to_csv('BASE_CONTROL.csv',sep='|', index=False)

In [72]:
BASE_GESTION.to_csv('BASE_GESTION.csv',sep='|', index=False)

In [73]:
#val nulls

In [74]:
reales_total['Total'].head(2)

KeyError: 'Total'

In [ ]:
reales_total['Total'].head(2)

In [ ]:
#reales_total_C=reales_total['Total'].to_numpy()

In [75]:
reales_total.isna().sum()

predicciones    0
Clase           0
dtype: int64

In [ ]:
pred_C.sum()

In [ ]:
len(pred_C)

# Para la prediccion no se necesita medir exactitud

In [49]:
for i in range (1,10):
    print(i/10)
    pred_C=predicciones_total>=i/10
    print('pred_C',len(pred_C))
    print('reales',len(reales_total))
    print(f'precision: {round(precision_score(reales_total_C, pred_C),5)}')
    print(f'recall: {round(recall_score(reales_total_C, pred_C),5)}')
    print(f'AUC: {round(roc_auc_score(reales_total_C,pred_C),5)}')
    print(f'LIFT: {round(lift_score(reales_total_C,pred_C),5)}') 
    matrizConfucion = confusion_matrix(reales_total_C, pred_C)
    print(matrizConfucion)

0.1
pred_C 5000000
reales 5000000
precision: 0.0


Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.


recall: 0.0


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
def plot_Lift_curve(y_val, y_pred, step=0.01):    
    #Define an auxiliar dataframe to plot the curve
    aux_lift = pd.DataFrame()
    #Create a real and predicted column for our new DataFrame and assign values
    aux_lift['real'] = y_val
    aux_lift['predicted'] = y_pred
    #Order the values for the predicted probability column:
    aux_lift.sort_values('predicted',ascending=False,inplace=True)
    
    #Create the values that will go into the X axis of our plot
    x_val = np.arange(step,1+step,step)
    #Calculate the ratio of ones in our data
    ratio_ones = aux_lift['real'].sum() / len(aux_lift)
    #Create an empty vector with the values that will go on the Y axis our our plot
    y_v = []
    
    #Calculate for each x value its correspondent y value
    for x in x_val:
        num_data = int(np.ceil(x*len(aux_lift))) #The ceil function returns the closest integer bigger than our number 
        data_here = aux_lift.iloc[:num_data,:]   # ie. np.ceil(1.4) = 2
        ratio_ones_here = data_here['real'].sum()/len(data_here)
        y_v.append(ratio_ones_here / ratio_ones)
           
   #Plot the figure
    fig, axis = plt.subplots()
    fig.figsize = (40,40)
    axis.plot(x_val, y_v, 'g-', linewidth = 3, markersize = 5)
    axis.plot(x_val, np.ones(len(x_val)), 'k-')
    axis.set_xlabel('Proportion of sample')
    axis.set_ylabel('Lift')
    plt.title('Lift Curve')
    plt.show()

In [ ]:
plot_Lift_curve(reales_total_C,predicciones_total)

In [ ]:
type(reales_total)

In [ ]:
len(predicciones_total)

In [ ]:
len(reales_total)

###  Corte para separar los casos de cambio de equipo 0.5